In [1]:
import requests
import json
import pandas as pd
url = 'https://raw.githubusercontent.com/Nyanza/lol-analytics/master/scraping/champions.json';
g = requests.get(url)
host = "http://localhost:8000"
path = 'http://ddragon.leagueoflegends.com/cdn/img/champion'
square_path = 'http://ddragon.leagueoflegends.com/cdn/6.24.1/img/champion'

In [2]:
from os import listdir
from os.path import isfile, join
json_path = './champstats/'
json_files = [f for f in listdir(json_path) if isfile(join(json_path, f))]

def load_json(path, file):
    with open(path+file) as data_file:    
        return json.load(data_file)

In [3]:
import operator
sorted_champ = sorted(g.json().items(), key=operator.itemgetter(0))

In [4]:
average = {}
def insert_to_db(name, info, counter):
    key = info['key'] #get champ key
    name = info['name'] #get champ name
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'} #set headers
    info['metrics'] = {} #init metrics
    metrics = load_json(json_path, json_files[counter])
    for league in metrics:
        for lane in metrics[league]:
            if 'http' not in lane:
                for x in metrics[league][lane]["build"]:
                    x["img"] = x["src"]
                    x.pop('src', None)
                if "counters" in metrics[league][lane]:
                    for z in metrics[league][lane]["counters"]:
                        for weak_strong in metrics[league][lane]["counters"][z]:
                            print(weak_strong)
                            weak_strong["win_rate"] = float(weak_strong["win_rate"].replace("%", ""))
                metrics[league][lane]["radar_chart"] = {}
                for y in list(metrics[league][lane]["stats"]):
                    new_key = y.replace(" ", "_").lower()
                    str_num = str(round(float(metrics[league][lane]["stats"][y].replace("%", "")), 2))
                    if str_num[-1] == "0":
                        str_num = int(str_num[0:len(str_num)-2])
                    else:
                        str_num = round(float(metrics[league][lane]["stats"][y].replace("%", "")), 2)
                    metrics[league][lane]["stats"][new_key] = str_num
                    z = metrics[league][lane]["stats"].pop(y, None)
                    metrics[league][lane]["radar_chart"][new_key] = round((float(z.replace("%", "")) / average[league][lane][new_key]["total"]) * 10, 5)
    #print(json.dumps(metrics))
    skill_images = metrics.pop('skillImages', None)
    info['metrics'] = metrics
    default_metrics = {}
    for lane in metrics["platplus"]:
        default_lane = lane
        break
    default_metrics['win_rate'] = metrics["platplus"][default_lane]["stats"]['win_rate']
    default_metrics['play_rate'] = metrics["platplus"][default_lane]["stats"]['pick_rate']
    default_metrics['ban_rate'] = metrics["platplus"][default_lane]["stats"]['ban_rate']
    info['defaultMetrics'] = default_metrics
    info['squareImg'] = square_path + '/' + key + '.png'
    skins = info['skins']
    for skin in skins:
        ext = key + '_' + str(skin['num'])  + '.jpg'
        skin['loading'] = path + '/loading/' + ext
        skin['splash'] = path + '/splash/' + ext
    info['skins'] = skins
    counter = 0
    spells = {'q': {}, 'w': {}, 'e': {}, 'r': {}}
    for spell in info['spells']:
        skill = 'qwer'[counter]
        spells[skill]['img'] = skill_images[counter]
        spells[skill]['description'] = spell['description']
        spells[skill]['name'] = spell['name']
        counter += 1
    info['spells'] = spells
    p = requests.post(host+"/api/v1/champions/", data=json.dumps(info), headers=headers)
    print(name)
    print(p)

def get_average():
    key = info['key'] #get champ key
    name = info['name'] #get champ name
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'} #set headers
    info['metrics'] = {} #init metrics
    metrics = load_json(json_path, json_files[counter])
    for league in metrics:
        for lane in metrics[league]:
            if 'http' not in lane:
                for y in list(metrics[league][lane]["stats"]):
                    new_key = y.replace(" ", "_").lower()
                    metrics[league][lane]["stats"][new_key] = metrics[league][lane]["stats"][y]
                    z = metrics[league][lane]["stats"].pop(y, None)
                    if league not in average:
                        average[league] = {}
                    if lane not in average[league]:
                        average[league][lane] = {}
                    if new_key not in average[league][lane]:
                        average[league][lane][new_key] = {}
                        average[league][lane][new_key]["average"] = float(z.replace("%",""))
                        average[league][lane][new_key]["total"] = float(z.replace("%",""))
                    else:
                        average[league][lane][new_key]["average"] = (average[league][lane][new_key]["average"] + float(z.replace("%","")))/2
                        average[league][lane][new_key]["total"] += float(z.replace("%", ""))
counter = 0
for name, info in sorted_champ:
    get_average()
    counter += 1
counter = 0
for name, info in sorted_champ:
    insert_to_db(name, info, counter)
    counter += 1
    if counter < 1:
        break

{'name': 'Teemo', 'win_rate': '38.46%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Teemo.png'}
{'name': 'Kennen', 'win_rate': '41.33%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Kennen.png'}
{'name': 'Vladimir', 'win_rate': '41.38%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Vladimir.png'}
{'name': 'Pantheon', 'win_rate': '44.87%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Pantheon.png'}
{'name': 'Singed', 'win_rate': '45.45%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Singed.png'}
{'name': 'Rumble', 'win_rate': '60.49%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Rumble.png'}
{'name': 'Tryndamere', 'win_rate': '59.04%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Tryndamere.png'}
{'name': 'Nasus', 'win_rate': '56.86%', 'img': 'https://ddragon.leagueoflegends.com/cdn/7.19.1/img/champion/Nasus.png'}
{'name': 'Ga

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/v1/champions/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10d9c0eb8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [5]:
str_num = "52.0"
if str_num[-1] == "0":
    str_num = str_num[0:len(str_num)-2]
str_num

'52'

In [6]:
url = 'http://localhost:8000/api/champions'
g = requests.get(url)
#get champion image data
path = './assets/champion/'
for champ in g.json():
    print(champ)
    fullChamp = requests.get(url+'/'+champ['name']).json()[0]
    # check if img exists in ./assets/sprite
    if not os.path.isfile(path + fullChamp['img']):
        # if not, download
        try:
            urllib.request.urlretrieve("http://ddragon.leagueoflegends.com/cdn/7.5.2/img/champion/" + fullChamp['img'], path + fullChamp['img'])
        except:
            continue

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/champions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10dd16fd0>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [7]:
url = 'http://localhost:8000/api/champions'
g = requests.get(url)
#get champion image data
path = './assets/loading/'
img_url = 'http://ddragon.leagueoflegends.com/cdn/img/champion/loading/' #key_skinNum.jpg
for champ in g.json():
    fullChamp = requests.get(url+'/'+champ['name']).json()[0]
    # if not, download
    key = fullChamp['key']
    for skin in fullChamp['skins']:
        ext = key + '_' + str(skin['num'])  + '.jpg'
        print(img_url + ext)
        # check if img exists in ./assets/sprite
        if not os.path.isfile(path + ext):
            try:
                urllib.request.urlretrieve(img_url + ext, path + ext)
            except:
                continue

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/champions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10de508d0>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [8]:
import requests
import json
#init image data
host = "http://localhost:8000"
path = '/assets'
champs = requests.get(host+'/api/v1/champions').json()
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
for champ in champs:
    fullChamp = requests.get(host+'/api/v1/champions/'+champ['name']).json()[0]
    # if not, download
    key = fullChamp['key']
    urls = { 'square': '', 'loading': [], 'splash': []}
    urls['square'] = host + path + '/square/' + key + '.png'
    for skin in fullChamp['skins']:
        ext = key + '_' + str(skin['num'])  + '.jpg'
        urls['loading'].append(host + path + '/loading/' + ext)
        urls['splash'].append(host + path + '/loading/' + ext)
    p = requests.post(host+'/api/v1/assets/images/champion', data=json.dumps({'name':key, 'loading': urls['loading'], 'square': urls['square'], 'splash': urls['splash']}), headers=headers)


ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /api/v1/champions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10de67b70>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [13]:
coins = [4, 3, 3, 3, 3, 4, 4]
def find_genuine(l):
    g = []
    for i in range(0, len(l)):
        for j in range(i, len(l)):
            if l[i] > l[j]:
                g.append(i)
            elif l[i] < l[j]:
                g.append(j)
            #else:
              #  if g: 
                    #g.pop()
    print(g)
find_genuine(coins)

[0, 0, 0, 0, 5, 6, 5, 6, 5, 6, 5, 6]


In [14]:
round(45.34531, 1)

45.3

In [15]:
int(0500)

SyntaxError: invalid token (<ipython-input-15-4a337a9e38cd>, line 1)